In [124]:
import pandas as pd
import os
import sys
import json
import re

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [99]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )

# Define data output path
transcript_path = os.path.join(git_dir, 'data', 'pm-transcripts-processed/aggregated-spellfixed.csv')
df = pd.read_csv(transcript_path)
df['date'] = pd.to_datetime(df.date)

In [73]:
df['content'] = df['content'].str.replace(u'\xa0', u' ')
df['content'] = df['content'].str.replace(' \]\]\>', '')

In [88]:
pms  = ['Abbott, Tony', 'Turnbull, Malcolm','Howard, John', 'Gillard, Julia','Rudd, Kevin', 'Morrison, Scott']

In [113]:
blob = TextBlob(text)

In [130]:
blob.sentiment

Sentiment(polarity=0.23000841750841752, subjectivity=0.3961560044893378)